# Crash Dummies- Phase 3 Project 
Angela Joseph-Pauline

In [37]:
#imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#this allows plotsto appear directly in the notebook
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor

### import DataFrame and clean data

In [38]:
df = pd.read_csv("Traffic_Crashes_-_Crashes.csv")
df.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,JC343143,NaN,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,JA329216,NaN,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,JD292400,NaN,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,3.0,0.0,10,6,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008)
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,JD293602,NaN,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,...,0.0,0.0,3.0,0.0,1,7,7,41.802119,-87.622115,POINT (-87.622114914961 41.802118543011)
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,JD290451,NaN,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,14,4,7,NaN,NaN,NaN


In [39]:
#examine data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496491 entries, 0 to 496490
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                496491 non-null  object 
 1   RD_NO                          492845 non-null  object 
 2   CRASH_DATE_EST_I               37328 non-null   object 
 3   CRASH_DATE                     496491 non-null  object 
 4   POSTED_SPEED_LIMIT             496491 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         496491 non-null  object 
 6   DEVICE_CONDITION               496491 non-null  object 
 7   WEATHER_CONDITION              496491 non-null  object 
 8   LIGHTING_CONDITION             496491 non-null  object 
 9   FIRST_CRASH_TYPE               496491 non-null  object 
 10  TRAFFICWAY_TYPE                496491 non-null  object 
 11  LANE_CNT                       198965 non-null  float64
 12  ALIGNMENT                     

In [40]:
#change all column names to lower case
df.columns = [x.lower() for x in df.columns]
df.head()

,crash_record_id,rd_no,crash_date_est_i,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,...,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,JC343143,NaN,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,JA329216,NaN,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,JD292400,NaN,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,3.0,0.0,10,6,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008)
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,JD293602,NaN,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,...,0.0,0.0,3.0,0.0,1,7,7,41.802119,-87.622115,POINT (-87.622114914961 41.802118543011)
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,JD290451,NaN,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,14,4,7,NaN,NaN,NaN


In [41]:
#Check for null values
print(df.isnull().sum())

crash_record_id                       0
rd_no                              3646
crash_date_est_i                 459163
crash_date                            0
posted_speed_limit                    0
traffic_control_device                0
device_condition                      0
weather_condition                     0
lighting_condition                    0
first_crash_type                      0
trafficway_type                       0
lane_cnt                         297526
alignment                             0
roadway_surface_cond                  0
road_defect                           0
report_type                       12156
crash_type                            0
intersection_related_i           384435
not_right_of_way_i               473049
hit_and_run_i                    349519
damage                                0
date_police_notified                  0
prim_contributory_cause               0
sec_contributory_cause                0
street_no                             0


In [42]:
#Drop null values
df=df.dropna(axis=1)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496491 entries, 0 to 496490
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   crash_record_id          496491 non-null  object
 1   crash_date               496491 non-null  object
 2   posted_speed_limit       496491 non-null  int64 
 3   traffic_control_device   496491 non-null  object
 4   device_condition         496491 non-null  object
 5   weather_condition        496491 non-null  object
 6   lighting_condition       496491 non-null  object
 7   first_crash_type         496491 non-null  object
 8   trafficway_type          496491 non-null  object
 9   alignment                496491 non-null  object
 10  roadway_surface_cond     496491 non-null  object
 11  road_defect              496491 non-null  object
 12  crash_type               496491 non-null  object
 13  damage                   496491 non-null  object
 14  date_police_notified

In [44]:
#Confirm nulls have been dropped
print(df.isnull().sum())

crash_record_id            0
crash_date                 0
posted_speed_limit         0
traffic_control_device     0
device_condition           0
weather_condition          0
lighting_condition         0
first_crash_type           0
trafficway_type            0
alignment                  0
roadway_surface_cond       0
road_defect                0
crash_type                 0
damage                     0
date_police_notified       0
prim_contributory_cause    0
sec_contributory_cause     0
street_no                  0
num_units                  0
crash_hour                 0
crash_day_of_week          0
crash_month                0
dtype: int64


### Assign Driver responsibility

In [45]:
#check value count
df['prim_contributory_cause'].value_counts().to_dict()

{'UNABLE TO DETERMINE': 184177,
 'FAILING TO YIELD RIGHT-OF-WAY': 54473,
 'FOLLOWING TOO CLOSELY': 52427,
 'NOT APPLICABLE': 26650,
 'IMPROPER OVERTAKING/PASSING': 23572,
 'IMPROPER BACKING': 21669,
 'FAILING TO REDUCE SPEED TO AVOID CRASH': 21422,
 'IMPROPER LANE USAGE': 19136,
 'IMPROPER TURNING/NO SIGNAL': 16439,
 'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE': 15567,
 'DISREGARDING TRAFFIC SIGNALS': 9036,
 'WEATHER': 8529,
 'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER': 6205,
 'DISREGARDING STOP SIGN': 5500,
 'DISTRACTION - FROM INSIDE VEHICLE': 3640,
 'EQUIPMENT - VEHICLE CONDITION': 3121,
 'PHYSICAL CONDITION OF DRIVER': 2915,
 'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)': 2911,
 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)': 2647,
 'DRIVING ON WRONG SIDE/WRONG WAY': 2346,
 'DISTRACTION - FROM OUTSIDE VEHICLE': 2185,
 'EXCEEDING AUTHORIZED SPEED LIMIT': 1982,
 'EXCEEDING SAFE SPEED FOR CONDITIONS': 1684,
 'ROAD ENGINEER

In [46]:
#assigning values
num= {'UNABLE TO DETERMINE': 0,
 'FAILING TO YIELD RIGHT-OF-WAY': 1,
 'FOLLOWING TOO CLOSELY': 1,
 'NOT APPLICABLE': 0,
 'IMPROPER OVERTAKING/PASSING': 23572,
 'IMPROPER BACKING': 21669,
 'FAILING TO REDUCE SPEED TO AVOID CRASH': 21422,
 'IMPROPER LANE USAGE': 19136,
 'IMPROPER TURNING/NO SIGNAL': 16439,
 'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE': 15567,
 'DISREGARDING TRAFFIC SIGNALS': 9036,
 'WEATHER': 0,
 'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER': 6205,
 'DISREGARDING STOP SIGN': 5500,
 'DISTRACTION - FROM INSIDE VEHICLE': 3640,
 'EQUIPMENT - VEHICLE CONDITION': 0,
 'PHYSICAL CONDITION OF DRIVER': 2915,
 'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)': 0,
 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)': 2647,
 'DRIVING ON WRONG SIDE/WRONG WAY': 2346,
 'DISTRACTION - FROM OUTSIDE VEHICLE': 2185,
 'EXCEEDING AUTHORIZED SPEED LIMIT': 1982,
 'EXCEEDING SAFE SPEED FOR CONDITIONS': 1684,
 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS': 0,
 'ROAD CONSTRUCTION/MAINTENANCE': 0,
 'DISREGARDING OTHER TRAFFIC SIGNS': 1068,
 'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST': 0,
 'CELL PHONE USE OTHER THAN TEXTING': 697,
 'DISREGARDING ROAD MARKINGS': 677,
 'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)': 551,
 'ANIMAL': 0,
 'TURNING RIGHT ON RED': 349,
 'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)': 235,
 'TEXTING': 215,
 'DISREGARDING YIELD SIGN': 187,
 'RELATED TO BUS STOP': 168,
 'BICYCLE ADVANCING LEGALLY ON RED LIGHT': 66,
 'PASSING STOPPED SCHOOL BUS': 64,
 'OBSTRUCTED CROSSWALKS': 33,
 'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT': 19}

In [47]:
#assign 1- to driver responsibility
for key in num.keys():
    if num[key]:
        num[key]=1

In [48]:
num

{'UNABLE TO DETERMINE': 0,
 'FAILING TO YIELD RIGHT-OF-WAY': 1,
 'FOLLOWING TOO CLOSELY': 1,
 'NOT APPLICABLE': 0,
 'IMPROPER OVERTAKING/PASSING': 1,
 'IMPROPER BACKING': 1,
 'FAILING TO REDUCE SPEED TO AVOID CRASH': 1,
 'IMPROPER LANE USAGE': 1,
 'IMPROPER TURNING/NO SIGNAL': 1,
 'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE': 1,
 'DISREGARDING TRAFFIC SIGNALS': 1,
 'WEATHER': 0,
 'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER': 1,
 'DISREGARDING STOP SIGN': 1,
 'DISTRACTION - FROM INSIDE VEHICLE': 1,
 'EQUIPMENT - VEHICLE CONDITION': 0,
 'PHYSICAL CONDITION OF DRIVER': 1,
 'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)': 0,
 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)': 1,
 'DRIVING ON WRONG SIDE/WRONG WAY': 1,
 'DISTRACTION - FROM OUTSIDE VEHICLE': 1,
 'EXCEEDING AUTHORIZED SPEED LIMIT': 1,
 'EXCEEDING SAFE SPEED FOR CONDITIONS': 1,
 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS': 0,
 'ROAD CONSTRUCTION/MAINTENANCE': 0,
 'DISREGAR

In [49]:
##apply new column to DataFrame
df['target']= df.prim_contributory_cause.map(num)

In [50]:
#check that it is in the DataFrame
df.head()

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,damage,date_police_notified,prim_contributory_cause,sec_contributory_cause,street_no,num_units,crash_hour,crash_day_of_week,crash_month,target
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,STRAIGHT AND LEVEL,...,"OVER $1,500",07/10/2019 06:16:00 PM,IMPROPER BACKING,UNABLE TO DETERMINE,2158,2,17,4,7,1
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,"OVER $1,500",06/30/2017 04:01:00 PM,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,8301,2,16,6,6,1
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,STRAIGHT AND LEVEL,...,"OVER $1,500",07/10/2020 10:25:00 AM,FAILING TO YIELD RIGHT-OF-WAY,FAILING TO YIELD RIGHT-OF-WAY,1632,3,10,6,7,1
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,$500 OR LESS,07/11/2020 08:30:00 AM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,110,2,1,7,7,0
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRIVEWAY,STRAIGHT AND LEVEL,...,"OVER $1,500",07/08/2020 02:15:00 PM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,412,2,14,4,7,0


## Train and Model data

In [51]:
#Check that data is uniform
df.dtypes

crash_record_id            object
crash_date                 object
posted_speed_limit          int64
traffic_control_device     object
device_condition           object
weather_condition          object
lighting_condition         object
first_crash_type           object
trafficway_type            object
alignment                  object
roadway_surface_cond       object
road_defect                object
crash_type                 object
damage                     object
date_police_notified       object
prim_contributory_cause    object
sec_contributory_cause     object
street_no                   int64
num_units                   int64
crash_hour                  int64
crash_day_of_week           int64
crash_month                 int64
target                      int64
dtype: object

In [52]:
#drop unneeded columns
drop= ['crash_record_id',
'crash_date',
'first_crash_type',      
'crash_type',
'date_police_notified',
'prim_contributory_cause',
'sec_contributory_cause',
'street_no']
df= df.drop(drop, axis=1)

In [53]:
#train data
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [54]:
#seporate numerical features
num_feats = DummyClassifier(strategy= 'most_frequent')
X_train_numeric = X_train[['posted_speed_limit',   
                           'num_units',   
                             'crash_hour', 
                           'crash_day_of_week', 'crash_month', ]].copy()

In [55]:
#check cross validation score of numerical features
num_cross_val_score = cross_val_score(num_feats, X_train_numeric, y_train, cv=5)
num_cross_val_score

array([0.53746274, 0.53746274, 0.53744931, 0.53745653, 0.53745653])

This cross validation model shows using numerical features could be a good area to explore. The initial score is already over 50% so one could assume that using these features could have a similar return.  

In [56]:
log_reg_model = LogisticRegression()

In [57]:
#numeric logestical regession model
num_cross_val_score = cross_val_score(log_reg_model, X_train_numeric, y_train, cv=5)
num_cross_val_score

array([0.56822515, 0.56657357, 0.56857427, 0.5686759 , 0.56903844])

### Scaling Data

In [58]:
#fit and transform trained data into scaled data.
scaler= StandardScaler()
X_train_scaled=X_train_numeric.copy()
scaler.fit(X_train_scaled)
X_train_scaled=scaler.transform(X_train_scaled)

In [59]:
#scaled logestical regession model
num_cross_val_score = cross_val_score(log_reg_model, X_train_scaled, y_train, cv=5)
num_cross_val_score

array([0.56822515, 0.56657357, 0.56857427, 0.5686759 , 0.56903844])

The R^2 for all 5 of these samples can predict up to about 57% of the crashes in the Chicargo, IL area. As predicted from out first cross validation score was over 50%. The data still have a number of Categorical elements. The next steps will be to attempt to apply these elemets and attempt another Logistical Regression Model.

### OneHotEncode Categorical Data for logistical Regression

In [60]:
#find the Categorical Data
X_train.dtypes

posted_speed_limit         int64
traffic_control_device    object
device_condition          object
weather_condition         object
lighting_condition        object
trafficway_type           object
alignment                 object
roadway_surface_cond      object
road_defect               object
damage                    object
num_units                  int64
crash_hour                 int64
crash_day_of_week          int64
crash_month                int64
dtype: object

In [61]:
#fit and transform categoricals
categoricals = X_train.select_dtypes('object')
categorical_names = categoricals.columns
ohe = OneHotEncoder(sparse= False)
ohe.fit(categoricals)
categorical_ohe= ohe.transform(categoricals)

categorical_ohe=pd.DataFrame(categorical_ohe, columns= ohe.get_feature_names())

In [62]:
#drop and concat categorical data
X_train= X_train.reset_index()
X_train= X_train.drop(categorical_names, axis= 1)
X_train_ohe= pd.concat([X_train, categorical_ohe], axis= 1)


In [63]:
#confirm it worked
X_train_ohe.head()

,index,posted_speed_limit,num_units,crash_hour,crash_day_of_week,crash_month,x0_BICYCLE CROSSING SIGN,x0_DELINEATORS,x0_FLASHING CONTROL SIGNAL,x0_LANE USE MARKING,...,x7_DEBRIS ON ROADWAY,x7_NO DEFECTS,x7_OTHER,"x7_RUT, HOLES",x7_SHOULDER DEFECT,x7_UNKNOWN,x7_WORN SURFACE,x8_$500 OR LESS,"x8_$501 - $1,500","x8_OVER $1,500"
0,45171,35,2,8,2,1,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,198424,30,2,7,2,12,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,231653,10,2,13,7,10,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,19472,30,2,19,6,3,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,390007,30,2,5,4,6,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [64]:
#Preform a logistical model with ohe
log_reg_model = LogisticRegression()

print("Old:", num_cross_val_score)
print("New:", cross_val_score(log_reg_model, X_train_ohe, y_train, cv=5))

Old: [0.56822515 0.56657357 0.56857427 0.5686759  0.56903844]
New: [0.53746274 0.53746274 0.53744931 0.53745653 0.53745653]


The addition of the categorical data dropped the from about 57% to about 54%. The categorical data did not help the R^2. In an attempt to see if a different Model would yeild a better R^2 the next step will be the addition od the DecisionTreeRegressor Model. 

### DecisionTreeRegressor Model

In [34]:
dt_regressor = DecisionTreeRegressor(random_state=0)

print("Old:", num_cross_val_score)
print("New:", cross_val_score(dt_regressor, X_train_ohe, y_train, cv=5))


Old: [0.56822515 0.56657357 0.56857427 0.5686759  0.56903844]
New: [-0.80961237 -0.80667046 -0.81348092 -0.80439306 -0.81054065]


The DecisionTreeRegressor Model provided a negative R^2. The best model to use is the Logistical Model.

# # Conclusion

In conclusion, the best features for predicting a crash in Chicargo, IL were the numerical features. The features such as speed limit and time of day, about 57% chance of prediction. Categorical features could predict a crash more then half the time but it is still less then that of the numerical features, categorical features can predict the crashes about 53% of the time. Furthermore, The Logistical Model was a better choice than the DecisionTreeRegressor Model in this set of data to recieve the best results. 
